# Operations Analysis over employees.csv:
- Display schema and first few rows.
- Filter employees with a salary greater than 5000.
- Calculate the average salary per department.
- Find the employee with the highest salary in each department.
- Add a new column [Salary Increase] which increases salaries by 10% 
for employees with more than 5 years of experience.
- Sort the employees by experience in descending order and show the top 5.

In [50]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max, when, round

spark = SparkSession.builder.appName("EmployeeAnalysis").getOrCreate()

dataframe = spark.read.csv("employees.csv", header=True, inferSchema=True)

## Display schema and first few rows

In [51]:
dataframe.printSchema()     
dataframe.show(10)

root
 |-- EmployeeID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Experience (Years): integer (nullable = true)

+----------+-------+----------+------+------------------+
|EmployeeID|   Name|Department|Salary|Experience (Years)|
+----------+-------+----------+------+------------------+
|      1001|  Frank|        IT|  8231|                20|
|      1002|  Alice|        HR|  3716|                23|
|      1003|  Alice| Marketing|  9334|                11|
|      1004|  David|     Sales| 14785|                 8|
|      1005|   Kara|Operations|  8301|                13|
|      1006|Charlie|Operations|  5720|                28|
|      1007|    Eve|        IT|  4801|                 4|
|      1008|   Liam|   Finance|  9243|                15|
|      1009|    Bob|   Finance| 11775|                 5|
|      1010|    Bob|Operations|  6456|                27|
+----------+-------+---------

## Filter employees with a salary greater than 5000

In [52]:
Salary5000_dataframe = dataframe.filter(col("salary") > 5000)
Salary5000_dataframe.show()

+----------+-------+----------+------+------------------+
|EmployeeID|   Name|Department|Salary|Experience (Years)|
+----------+-------+----------+------+------------------+
|      1001|  Frank|        IT|  8231|                20|
|      1003|  Alice| Marketing|  9334|                11|
|      1004|  David|     Sales| 14785|                 8|
|      1005|   Kara|Operations|  8301|                13|
|      1006|Charlie|Operations|  5720|                28|
|      1008|   Liam|   Finance|  9243|                15|
|      1009|    Bob|   Finance| 11775|                 5|
|      1010|    Bob|Operations|  6456|                27|
|      1011| Hannah|        IT|  6457|                15|
|      1012|Charlie|        HR| 11244|                29|
|      1013|  Alice| Marketing|  8875|                16|
|      1014|  Frank|Operations|  7422|                28|
|      1016|  David|   Finance|  7087|                24|
|      1018|    Bob|     Sales| 11012|                17|
|      1019|  

## Calculate the average salary per department

In [53]:
AverageSalary_Department_dataframe = dataframe.groupBy("department").agg(round(avg("salary"),2).alias("average_salary"))
AverageSalary_Department_dataframe.show()

+----------+--------------+
|department|average_salary|
+----------+--------------+
|     Sales|       8680.87|
|        HR|       8596.45|
|   Finance|        9233.8|
| Marketing|       8708.29|
|        IT|       8790.34|
|Operations|       8879.69|
+----------+--------------+



## Find the employee with the highest salary in each department

In [54]:
# Departments and their highest salary
department_max_salary = dataframe.groupBy("department").agg(max("salary").alias("highest_salary"))

# Used to join the department_max_salary dataframe with the original dataframe (to have the employee name)
employees_with_highest_salary = dataframe.alias("employee").join(
    department_max_salary.alias("dept_max"),
    (col("employee.department") == col("dept_max.department")) & 
    (col("employee.salary") == col("dept_max.highest_salary")),
    "inner"
).select(
    col("employee.name"),
    col("employee.department"), 
    col("employee.salary").alias("highest_salary")
)

employees_with_highest_salary.show()

+-----+----------+--------------+
| name|department|highest_salary|
+-----+----------+--------------+
|David|   Finance|         14928|
|  Bob|     Sales|         14974|
|Alice|        IT|         14859|
|Frank|        HR|         14999|
|  Eve|Operations|         14978|
|Alice| Marketing|         14938|
+-----+----------+--------------+



## Add a new column [Salary Increase] which increases salaries by 10% for employees with more than 5 years of experience

In [55]:
increased_salary_dataframe = dataframe.withColumn(
    "salary_increase",
    round(
    when(col("Experience (Years)") > 5, col("salary") * 1.1).otherwise(col("salary")), 2)
) 

increased_salary_dataframe.show()

+----------+-------+----------+------+------------------+---------------+
|EmployeeID|   Name|Department|Salary|Experience (Years)|salary_increase|
+----------+-------+----------+------+------------------+---------------+
|      1001|  Frank|        IT|  8231|                20|         9054.1|
|      1002|  Alice|        HR|  3716|                23|         4087.6|
|      1003|  Alice| Marketing|  9334|                11|        10267.4|
|      1004|  David|     Sales| 14785|                 8|        16263.5|
|      1005|   Kara|Operations|  8301|                13|         9131.1|
|      1006|Charlie|Operations|  5720|                28|         6292.0|
|      1007|    Eve|        IT|  4801|                 4|         4801.0|
|      1008|   Liam|   Finance|  9243|                15|        10167.3|
|      1009|    Bob|   Finance| 11775|                 5|        11775.0|
|      1010|    Bob|Operations|  6456|                27|         7101.6|
|      1011| Hannah|        IT|  6457|

## Sort the employees by experience in descending order and show the top 5

In [56]:
more_experient_employees = dataframe.orderBy(col("Experience (Years)").desc())
more_experient_employees.show(5)

+----------+-------+----------+------+------------------+
|EmployeeID|   Name|Department|Salary|Experience (Years)|
+----------+-------+----------+------+------------------+
|      1090|  Alice| Marketing|  9467|                30|
|      1139|   Liam|     Sales|  5967|                30|
|      1111|    Ian|     Sales|  6763|                30|
|      1112|   Kara|        IT|  5926|                30|
|      1081|Charlie|     Sales| 10647|                30|
+----------+-------+----------+------+------------------+
only showing top 5 rows

